In [1]:
#jupyter如果無法用，去terminal試

import multiprocessing as mp
#import threading as td

def job(a,b):
    print('aaa')

if __name__=='__main__': #要加這個
    #t1 = td.Thread(target=job, args=(a,b))  #加thread
    p1 = mp.Process(target=job, args=(1,2)) #加Process,用法跟thread一樣
    p1.start()
    p1.join()


aaa


In [3]:
import multiprocessing as mp

def job(q):
    res = 0
    for i in range(1000):
        res += i+i**2+i**3
    q.put(res) # queue

if __name__ == '__main__':
    q = mp.Queue()  # 定義儲存隊列, 跟thread一樣, Process無法返回數值
    p1 = mp.Process(target=job, args=(q,)) #args(q,) 逗點一定要
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print(res1)
    print(res2)
    print(res1+res2)

249833583000
249833583000
499667166000


In [4]:
#一般與多進程與多核心運算比較
import multiprocessing as mp
import threading as td
import time

def job(q):
    res = 0
    for i in range(1000000):
        res += i+i**2+i**3
    q.put(res) # queue

def multicore():
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print('multicore:' , res1+res2)

def normal():
    res = 0
    for _ in range(2):
        for i in range(1000000):
            res += i+i**2+i**3
    print('normal:', res)

def multithread():
    q = mp.Queue()
    t1 = td.Thread(target=job, args=(q,))
    t2 = td.Thread(target=job, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    res1 = q.get()
    res2 = q.get()
    print('multithread:', res1+res2)

if __name__ == '__main__':
    st = time.time()
    normal()
    st1= time.time()
    print('normal time:', st1 - st)
    multithread()
    st2 = time.time()
    print('multithread time:', st2 - st1)
    multicore()
    print('multicore time:', time.time()-st2)

normal: 499999666667166666000000
normal time: 1.641489028930664
multithread: 499999666667166666000000
multithread time: 1.6848890781402588
multicore: 499999666667166666000000
multicore time: 0.7779569625854492


In [8]:
# 進程池, 不用定義Process, 直接把job丟進ProcessPool, 讓python幫你處理分配multiprocessing
# function可以return值
import multiprocessing as mp

def job(x):
    return x*x

def multicore():
#     pool = mp.Pool()  # 默認拿所有核來當pool, 也可以指定只拿幾個核.
    pool = mp.Pool(processes=2) #上面job()可以有return
    
    res = pool.map(job, range(10))  # 用法一 map(), map所有
    print(res)
    
    res = pool.apply_async(job, (2,))  # 用法二 apply_async(), 指輸入一個值(但是要可以迭代...), 多值會報錯
    print(res.get())
    
    multi_res =[pool.apply_async(job, (i,)) for i in range(10)] # apply_async() + 迭代器使用
    print([res.get() for res in multi_res])

if __name__ == '__main__':
    multicore()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
4
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


# 全局變量

全局變量無法傳給各個Core,故無法共享戶傳

所以要共享記憶體才可以給不同core用

用法:
```python
import multiprocessing as mp
value = mp.Value('d', 1) #'d'為資料型態
array = mp.Arrar('i', [1,2,3]) #'i'為資料型態, 但array只能一維
```

| Type code | C Type             | Python Type       | Minimum size in bytes |
| --------- | ------------------ | ----------------- | --------------------- |
| `'b'`     | signed char        | int               | 1                     |
| `'B'`     | unsigned char      | int               | 1                     |
| `'u'`     | Py_UNICODE         | Unicode character | 2                     |
| `'h'`     | signed short       | int               | 2                     |
| `'H'`     | unsigned short     | int               | 2                     |
| `'i'`     | signed int         | int               | 2                     |
| `'I'`     | unsigned int       | int               | 2                     |
| `'l'`     | signed long        | int               | 4                     |
| `'L'`     | unsigned long      | int               | 4                     |
| `'q'`     | signed long long   | int               | 8                     |
| `'Q'`     | unsigned long long | int               | 8                     |
| `'f'`     | float              | float             | 4                     |
| `'d'`     | double             | float             | 8                     |

In [17]:
import multiprocessing as mp
import time

def job(v, num):
    for _ in range(5):
        time.sleep(1)
        v.value += num # v.value 拿出共享變數的值
        print(v.value, end="\n")
        
def multicore():
    v = mp.Value('i', 0) # 定義共享變數
    p1 = mp.Process(target=job, args=(v,1))
    p2 = mp.Process(target=job, args=(v,3)) # 設定不同的number看如何搶奪內存
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    
if __name__ == '__main__':
    multicore()

1
4
5
8
9
12
13
16
17
20


In [14]:
# 進程鎖
import multiprocessing as mp
import time

def job(v, num, l):
    l.acquire()  # 鎖住
    for _ in range(10):
        time.sleep(0.1)
        v.value += num  # 拿取共享變數得值
        print(v.value)
    l.release()  # 釋放

def multicore():
    l = mp.Lock()  # 定義進程鎖
    v = mp.Value('i', 0)
    p1 = mp.Process(target=job, args=(v, 1, l))  # 要把進程鎖傳入
    p2 = mp.Process(target=job, args=(v, 3, l))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__ == '__main__':
    multicore()


1
2
3
4
5
6
7
8
9
10
13
16
19
22
25
28
31
34
37
40
